In [ ]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome('/Applications/chromedriver')

import csv 

links = []

with open('links_wo_duplicates.csv') as csvfile:    
	csvReader = csv.reader(csvfile)    
	for row in csvReader:        
		links.append(row[0])

links.pop(0)	#remove first element which is the header	   
print(links)

dta = pd.DataFrame(columns=['Movie Name', 'Audience Score', 'Tomatometer Score', 'Rating', 'Genre',
                            'Original Language', 'Director', 'Producer', 'Writer', 'Release Date (Theaters)',
                           'Release Date (Streaming)', 'Box Office (Gross USA)', 'Runtime', 'Distributor',
                            'Sound Mix', 'Aspect Ratio', 'View the collection'])


In [ ]:

from tqdm import tqdm

for link in tqdm(links, desc='loading....'):
    driver.get(link)
    info_we = driver.find_elements(By.XPATH,'//score-board')
    movie_info = {'Movie Name':'','Audience Score':'','Tomatometer Score':''}
    movie_info['Movie Name'] = info_we[0].text.split('\n')[0]
    movie_info['Audience Score'] = info_we[0].get_attribute('audiencescore')
    movie_info['Tomatometer Score'] = info_we[0].get_attribute('tomatometerscore')
    webelement_list_val = []
    webelement_list_key = []
    webelement_list_key = driver.find_elements(By.XPATH,'//div[@data-qa="movie-info-item-label"]')
    webelement_list_val = driver.find_elements(By.XPATH,'//div[@data-qa="movie-info-item-value"]')
    key_list, val_list = [],[]
    for k,v in zip(webelement_list_key, webelement_list_val):
        key_list.append(k.text.strip(':'))
        val_list.append(v.text)
    info = dict(zip(key_list,val_list))

    total_info = {**movie_info,**info} 
    dta = dta.append(total_info, ignore_index=True)  

dta.to_csv('output/movies_dirty.csv', index=False)

dta['Audience Score'].loc[dta['Audience Score'] == ""] = -1

dta[['Tomatometer Score','Audience Score']]=dta[['Tomatometer Score','Audience Score']].astype(int)

dta['Release Date (Streaming)'] = pd.to_datetime(dta['Release Date (Streaming)'])

def func(x):
    if type(x) != float:
        li = x.split(' ')
        li.remove(li[-1])
        time = ' '.join(li).replace(',','')
        return time
dta['Release Date (Theaters)'] = dta['Release Date (Theaters)'].apply(func)
dta['Release Date (Theaters)'] = pd.to_datetime(dta['Release Date (Theaters)'])

dta['Rating'].loc[dta['Rating'].isnull()] = 'None'

dta['Rating'] = dta['Rating'].agg(lambda a : a.split(' ')[0])

dta.to_csv('output/cleanedMovies.csv', index=False)